In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [36]:
df =  pd.read_csv('train_6.csv', nrows = 100000)
df_test = pd.read_csv('test_6.csv')

In [37]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [38]:
df_test.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,08:24.0,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,08:24.0,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,53:44.0,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,12:12.0,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,12:12.0,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


In [39]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

df['year'] = df['pickup_datetime'].dt.year
df['month'] = df['pickup_datetime'].dt.month
df['day'] = df['pickup_datetime'].dt.day
df['hour'] = df['pickup_datetime'].dt.hour
df['minute'] = df['pickup_datetime'].dt.minute
df['Weekday'] = df['pickup_datetime'].dt.weekday

In [40]:
df_test['pickup_datetime'] = pd.to_datetime(df_test['pickup_datetime'])

df_test['year'] = df_test['pickup_datetime'].dt.year
df_test['month'] = df_test['pickup_datetime'].dt.month
df_test['day'] = df_test['pickup_datetime'].dt.day
df_test['hour'] = df_test['pickup_datetime'].dt.hour
df_test['minute'] = df_test['pickup_datetime'].dt.minute
df_test['Weekday'] = df_test['pickup_datetime'].dt.weekday

In [41]:
df.dropna(axis=0, how='any', inplace=True)

In [42]:
print(df.shape)
print(df_test.shape)

(100000, 14)
(9914, 13)


In [43]:
cond = True
cols = ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']
for col in cols:
    cond &= abs(df[col] - df[col].mean()) < 5

In [44]:
df = df[cond]

In [45]:
df.shape

(97920, 14)

In [46]:
# 0 = key, 2 = pickup_datetime
cols = [0, 2]
df = df.drop(df.columns[cols], axis=1)
cols = [0, 1]
df_test = df_test.drop(df_test.columns[cols], axis=1)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97920 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fare_amount        97920 non-null  float64
 1   pickup_longitude   97920 non-null  float64
 2   pickup_latitude    97920 non-null  float64
 3   dropoff_longitude  97920 non-null  float64
 4   dropoff_latitude   97920 non-null  float64
 5   passenger_count    97920 non-null  int64  
 6   year               97920 non-null  int64  
 7   month              97920 non-null  int64  
 8   day                97920 non-null  int64  
 9   hour               97920 non-null  int64  
 10  minute             97920 non-null  int64  
 11  Weekday            97920 non-null  int64  
dtypes: float64(5), int64(7)
memory usage: 9.7 MB


In [48]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pickup_longitude   9914 non-null   float64
 1   pickup_latitude    9914 non-null   float64
 2   dropoff_longitude  9914 non-null   float64
 3   dropoff_latitude   9914 non-null   float64
 4   passenger_count    9914 non-null   int64  
 5   year               9914 non-null   int64  
 6   month              9914 non-null   int64  
 7   day                9914 non-null   int64  
 8   hour               9914 non-null   int64  
 9   minute             9914 non-null   int64  
 10  Weekday            9914 non-null   int64  
dtypes: float64(4), int64(7)
memory usage: 852.1 KB


In [49]:
def calculate_distance(df):
    long_x1 = df['pickup_longitude']
    lat_y1 = df['pickup_latitude']
    long_x2 = df['dropoff_longitude']
    lat_y2 = df['dropoff_latitude']
    
    long_x1, lat_y1, long_x2, lat_y2 = map(np.radians, [long_x1, lat_y1, long_x2, lat_y2])
    diff_long = long_x2 - long_x1
    diff_lat = lat_y2 - lat_y1
    
    a = np.sin(diff_lat/2.0)**2 + np.cos(lat_y1) * np.cos(lat_y2) * np.sin(diff_long/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    #in Km
    return 6367*c 

In [50]:
df['distance'] = calculate_distance(df)
df_test['distance'] = calculate_distance(df_test)

In [51]:
df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,minute,Weekday,distance
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,17,26,0,1.030117
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,16,52,1,8.444828
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,0,35,3,1.388653
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,4,30,5,2.797513
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,7,51,1,1.997902


In [52]:
df_test.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,minute,Weekday,distance
0,-73.973320,40.763805,-73.981430,40.743835,1,2015,1,27,13,8,1,2.321801
1,-73.986862,40.719383,-73.998886,40.739201,1,2015,1,27,13,8,1,2.423830
2,-73.982524,40.751260,-73.979654,40.746139,1,2011,10,8,11,53,5,0.618239
3,-73.981160,40.767807,-73.990448,40.751635,1,2012,12,1,21,12,5,1.959801
4,-73.966046,40.789775,-73.988565,40.744427,1,2012,12,1,21,12,5,5.383919


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97920 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fare_amount        97920 non-null  float64
 1   pickup_longitude   97920 non-null  float64
 2   pickup_latitude    97920 non-null  float64
 3   dropoff_longitude  97920 non-null  float64
 4   dropoff_latitude   97920 non-null  float64
 5   passenger_count    97920 non-null  int64  
 6   year               97920 non-null  int64  
 7   month              97920 non-null  int64  
 8   day                97920 non-null  int64  
 9   hour               97920 non-null  int64  
 10  minute             97920 non-null  int64  
 11  Weekday            97920 non-null  int64  
 12  distance           97920 non-null  float64
dtypes: float64(6), int64(7)
memory usage: 10.5 MB


In [54]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pickup_longitude   9914 non-null   float64
 1   pickup_latitude    9914 non-null   float64
 2   dropoff_longitude  9914 non-null   float64
 3   dropoff_latitude   9914 non-null   float64
 4   passenger_count    9914 non-null   int64  
 5   year               9914 non-null   int64  
 6   month              9914 non-null   int64  
 7   day                9914 non-null   int64  
 8   hour               9914 non-null   int64  
 9   minute             9914 non-null   int64  
 10  Weekday            9914 non-null   int64  
 11  distance           9914 non-null   float64
dtypes: float64(5), int64(7)
memory usage: 929.6 KB


In [55]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,minute,Weekday,distance
count,97920.000000,97920.000000,97920.000000,97920.000000,97920.000000,97920.000000,97920.000000,97920.000000,97920.000000,97920.000000,97920.000000,97920.000000,97920.000000
mean,11.344036,-73.975148,40.751034,-73.974010,40.751465,1.674275,2011.742198,6.261417,15.662776,13.494301,29.606199,3.036264,3.372307
std,9.690390,0.043715,0.036154,0.039907,0.037153,1.300192,1.868337,3.452065,8.675838,6.504510,17.301472,1.949364,4.484154
min,-44.900000,-75.425388,38.754215,-75.354559,39.296307,0.000000,2009.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,-73.992267,40.736635,-73.991399,40.735749,1.000000,2010.000000,3.000000,8.000000,9.000000,15.000000,1.000000,1.254322
50%,8.500000,-73.982077,40.753534,-73.980436,40.753962,1.000000,2012.000000,6.000000,16.000000,14.000000,30.000000,3.000000,2.156278
75%,12.500000,-73.968288,40.767681,-73.965116,40.768485,2.000000,2013.000000,9.000000,23.000000,19.000000,45.000000,5.000000,3.926049
max,200.000000,-70.140793,43.183332,-72.854940,43.415190,6.000000,2015.000000,12.000000,31.000000,23.000000,59.000000,6.000000,376.209187


In [56]:
cols = ['fare_amount', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 
        'dropoff_latitude', 'passenger_count', 'distance']
Min = [1, -80, 40, -80, 40, 1, 1]
Max = [200, -70, 45, -70, 45, 10, 377]

for i in range(6):
    df = df[df[cols[i]] >= Min[i]]
    df = df[df[cols[i]] <= Max[i]]

In [57]:
df.shape

(97528, 13)

In [58]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,minute,Weekday,distance
count,97528.000000,97528.000000,97528.000000,97528.000000,97528.000000,97528.000000,97528.000000,97528.000000,97528.000000,97528.000000,97528.000000,97528.000000,97528.000000
mean,11.353459,-73.975159,40.751213,-73.974020,40.751593,1.680523,2011.743397,6.264591,15.662866,13.492894,29.603252,3.036810,3.350141
std,9.697196,0.041655,0.033016,0.039669,0.035153,1.298639,1.871294,3.451759,8.676806,6.505703,17.300422,1.949515,4.015454
min,2.500000,-75.425388,40.121653,-75.354559,40.164927,1.000000,2009.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,-73.992267,40.736656,-73.991397,40.735770,1.000000,2010.000000,3.000000,8.000000,9.000000,15.000000,1.000000,1.254465
50%,8.500000,-73.982077,40.753538,-73.980438,40.753965,1.000000,2012.000000,6.000000,16.000000,14.000000,30.000000,3.000000,2.156397
75%,12.500000,-73.968291,40.767685,-73.965135,40.768478,2.000000,2013.000000,9.000000,23.000000,19.000000,45.000000,5.000000,3.926049
max,200.000000,-72.856968,43.183332,-72.854940,43.415190,6.000000,2015.000000,12.000000,31.000000,23.000000,59.000000,6.000000,270.427843


In [59]:
df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day,hour,minute,Weekday,distance
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,17,26,0,1.030117
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,16,52,1,8.444828
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,0,35,3,1.388653
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,4,30,5,2.797513
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,7,51,1,1.997902


In [60]:
df.shape

(97528, 13)

In [61]:
Y_train = df['fare_amount']

cols = [0, 1, 2, 3, 4, 10]
X_train = df.drop(columns=df.columns[cols],  axis=1)

cols = [0, 1, 2, 3, 9]
X_test = df_test.drop(columns=df_test.columns[cols],  axis=1)


In [62]:
X_train

,passenger_count,year,month,day,hour,Weekday,distance
0,1,2009,6,15,17,0,1.030117
1,1,2010,1,5,16,1,8.444828
2,2,2011,8,18,0,3,1.388653
3,1,2012,4,21,4,5,2.797513
4,1,2010,3,9,7,1,1.997902
...,...,...,...,...,...,...,...
99995,5,2013,9,24,7,1,1.606908
99996,1,2014,5,15,12,3,1.692369
99997,1,2015,2,19,17,3,0.424716
99998,4,2009,10,10,23,5,1.158389


In [63]:
Y_train

0         4.5
1        16.9
2         5.7
3         7.7
4         5.3
         ... 
99995     9.0
99996     6.0
99997     5.0
99998     6.9
99999     5.7
Name: fare_amount, Length: 97528, dtype: float64

In [64]:
X_test

,passenger_count,year,month,day,hour,Weekday,distance
0,1,2015,1,27,13,1,2.321801
1,1,2015,1,27,13,1,2.423830
2,1,2011,10,8,11,5,0.618239
3,1,2012,12,1,21,5,1.959801
4,1,2012,12,1,21,5,5.383919
...,...,...,...,...,...,...,...
9909,6,2015,5,10,12,6,2.123540
9910,6,2015,1,12,17,0,3.268916
9911,6,2015,4,19,20,6,19.171897
9912,6,2015,1,31,1,5,8.338248


In [65]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pickup_longitude   9914 non-null   float64
 1   pickup_latitude    9914 non-null   float64
 2   dropoff_longitude  9914 non-null   float64
 3   dropoff_latitude   9914 non-null   float64
 4   passenger_count    9914 non-null   int64  
 5   year               9914 non-null   int64  
 6   month              9914 non-null   int64  
 7   day                9914 non-null   int64  
 8   hour               9914 non-null   int64  
 9   minute             9914 non-null   int64  
 10  Weekday            9914 non-null   int64  
 11  distance           9914 non-null   float64
dtypes: float64(5), int64(7)
memory usage: 929.6 KB


In [66]:
classifier = RandomForestRegressor(random_state=10, n_estimators=1000, max_depth=8, oob_score=True)
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(X_test)

In [67]:
sample = pd.read_csv('sample_submission.csv')

In [68]:
sample.head()

,key,fare_amount
0,2015-01-27 13:08:24.0000002,11.35
1,2015-01-27 13:08:24.0000003,11.35
2,2011-10-08 11:53:44.0000002,11.35
3,2012-12-01 21:12:12.0000002,11.35
4,2012-12-01 21:12:12.0000003,11.35


In [69]:
keys = sample['key']

y_pred = pd.DataFrame(data = np.array(y_pred), columns = ['fare_amount'])
sub = pd.concat([keys, y_pred], axis=1)
sub.to_csv("submission.csv", index=False)